<a href="https://colab.research.google.com/github/uslumt/early_stopping_double_descent/blob/master/ATML_Reproduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/uslumt/early_stopping_double_descent.git

Cloning into 'early_stopping_double_descent'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 228 (delta 95), reused 162 (delta 49), pack-reused 0
Receiving objects: 100% (228/228), 2.21 MiB | 11.85 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [2]:
cd early_stopping_double_descent/

/content/early_stopping_double_descent


In [3]:
ls

code/    model_select.py  README.md         results/  train_two_layer.py
models/  notebooks/       requirements.txt  train.py


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install -r requirements.txt

In [6]:
!pip install opencv-python

In [ ]:
CONFIG_FILE="/content/early_stopping_double_descent/results/mcnn/cifar/ff_small_lr/config.json"

In [20]:
!mkdir datasets

mkdir: cannot create directory ‘datasets’: File exists


# The cells below transform the dataset from raw form to .npz form

In [21]:
import torchvision
trainset = torchvision.datasets.CIFAR10(root='./datasets', train=True, download=True)

Files already downloaded and verified


In [23]:
import glob
import os
import pickle

import cv2
import numpy as np


data_dir = './datasets/cifar-10-batches-py'

train_files = glob.glob(os.path.join(data_dir, 'data_batch*'))
test_files = glob.glob(os.path.join(data_dir, 'test_batch*'))

# all_files = all_files + test_files
assert train_files, 'File paths to load are empty. Please ensure downloaded files are in {}'.format(data_dir)
assert test_files, 'File paths to load are empty. Please ensure downloaded files are in {}'.format(data_dir)


def unpickle(file):
    with open(file, 'rb') as fo:
        dic = pickle.load(fo, encoding='bytes')
    return dic


# label
# 0:airplane, 1:automobile, 2:bird. 3:cat, 4:deer, 5:dog, 6:frog, 7:horse, 8:ship, 9:truck
X_train = []
y_train = []

X_test = []
y_test = []

print('Load compressed files - Train')
print(train_files)


for file in train_files:
    print('Processing', file)
    ret = unpickle(file)

    for i, arr in enumerate(ret[b'data']):
        img = np.reshape(arr, (3, 32, 32))
        img = img.transpose(1, 2, 0)
        X_train.append(img)
        y_train.append(ret[b'labels'][i])

X_train = np.array(X_train)
y_train = np.array(y_train)

print('Train files shape')
print(X_train.shape)

print('Train labeles shape')
print(y_train.shape)



print('Load compressed files - Test')
print(test_files)

for file in test_files:
    print('Processing', file)
    ret = unpickle(file)

    for i, arr in enumerate(ret[b'data']):
        img = np.reshape(arr, (3, 32, 32))
        img = img.transpose(1, 2, 0)
        X_test.append(img)
        y_test.append(ret[b'labels'][i])

X_test = np.array(X_test)
y_test = np.array(y_test)

print('Test files shape')
print(X_test.shape)

print('Test labeles shape')
print(y_test.shape)


np.savez(os.path.join('./datasets', 'cifar.npz'), X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

Load compressed files - Train
['./datasets/cifar-10-batches-py/data_batch_2', './datasets/cifar-10-batches-py/data_batch_4', './datasets/cifar-10-batches-py/data_batch_5', './datasets/cifar-10-batches-py/data_batch_3', './datasets/cifar-10-batches-py/data_batch_1']
Processing ./datasets/cifar-10-batches-py/data_batch_2
Processing ./datasets/cifar-10-batches-py/data_batch_4
Processing ./datasets/cifar-10-batches-py/data_batch_5
Processing ./datasets/cifar-10-batches-py/data_batch_3
Processing ./datasets/cifar-10-batches-py/data_batch_1
Train files shape
(50000, 32, 32, 3)
Train labeles shape
(50000,)
Load compressed files - Test
['./datasets/cifar-10-batches-py/test_batch']
Processing ./datasets/cifar-10-batches-py/test_batch
Test files shape
(10000, 32, 32, 3)
Test labeles shape
(10000,)


In [ ]:
!python3 train.py --config /content/early_stopping_double_descent/results/mcnn/cifar/ff_large_lr/config.json

train.py:792: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn('You have chosen a specific GPU. This will completely '
Use GPU: 0 for training
=> creating model 'resnet'
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Epoch: [0][0/391]	Time 2.628 (2.628)	Data 0.327 (0.327)	Loss 2.4719 (2.4719)	Acc@1 7.031 (7.031)	Acc@5 53.125 (53.125)
Test: [0/391]	Time 0.409 (0.409)	Loss 1.5799 (1.5799)	Acc@1 36.719 (36.719)	Acc@5 92.969 (92.969)
Test: [0/79]	Time 0.353 (0.353)	Loss 1.3684 (1.3684)	Acc@1 46.094 (46.094)	Acc@5 93.750 (93.750)
E

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
ls


sample_data/
